In [1]:
import os

import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "hawc.main.settings.local")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
import pandas as pd

from hawc.apps.assessment.models import Assessment
from hawc.apps.lit import models


In [3]:
# generate information from assessment needed to generate counts for the prisma plot

assessment_id = 2

# for reference-tag pairs, need references, tagtree, and list of reference-tag pairs. then join on each of those ids
assessment = Assessment.objects.get(id=assessment_id)
tag_tree = models.ReferenceFilterTag.get_all_tags(assessment_id)
references = assessment.references.all()
tags = models.ReferenceFilterTag.get_nested_tag_names(assessment_id)
tag_pairs: pd.DataFrame = models.ReferenceTags.objects.as_dataframe(assessment_id)



INFO 2024-08-30 16:00:49,926 hawc.apps.common.models cache set: reference-tagtree-assessment-2


In [4]:
tags_df = pd.DataFrame(tags.items(), columns=["tag_id", "tag_name"])
ref_df = pd.DataFrame(references.values())
ref_df = ref_df.rename(columns={"id":"reference_id"})

In [5]:
tag_pairs = tag_pairs.merge(tags_df, on="tag_id")
tag_pairs = tag_pairs.merge(ref_df, on="reference_id")

In [6]:
tag_pairs

,reference_id,tag_id,tag_name,assessment_id,title,authors_short,authors,year,journal,abstract,full_text_url,created,last_updated,block_id
0,5,12,Inclusion ➤ Human Study,2,Psycho-physiological effects of the terrorist ...,"Kawana N, Ishimatsu S, and Kanda K","Kawana N, Ishimatsu S, Kanda K",2001,Mil Med 2001; 166 (12 Suppl):23-6,The investigation describes the follow-up of t...,,2020-05-08 17:03:52.095604+00:00,2020-05-08 17:04:14.726133+00:00,None
1,6,13,Inclusion ➤ Animal Study,2,Effects of bisphenol A-related diphenylalkanes...,Letcher RJ et al.,"Letcher RJ, Sanderson JT, Bokkers A, Giesy JP,...",2005,Toxicol. Appl. Pharmacol. 2005; 209 (2):95-104,The present study investigated the effects of ...,,2020-05-08 17:03:52.203728+00:00,2020-05-08 17:04:17.766639+00:00,None
2,7,13,Inclusion ➤ Animal Study,2,An oral developmental neurotoxicity study of d...,Biesemeier JA et al.,"Biesemeier JA, Beck MJ, Silberberg H, Myers NR...",2011,Birth Defects Res. B Dev. Reprod. Toxicol. 201...,"<span class=""abstract_label"">BACKGROUND: </spa...",,2020-05-08 17:03:52.391195+00:00,2020-05-08 17:56:45.324222+00:00,None
3,8,12,Inclusion ➤ Human Study,2,Prenatal folic acid and risk of asthma in chil...,Crider KS et al.,"Crider KS, Cordero AM, Qi YP, Mulinare J, Dowl...",2013,Am. J. Clin. Nutr. 2013; 98 (5):1272-81,"<span class=""abstract_label"">BACKGROUND: </spa...",,2020-05-08 17:03:52.500862+00:00,2020-05-08 17:04:11.036574+00:00,None


In [7]:
# for reference-search pairs, need references, searches, and reference-search pairs
searches = assessment.literature_searches.all()
search_pairs = list(models.Reference.searches.through.objects.filter(reference_id__in=references).values("reference_id", "search_id"))


In [8]:
search_pairs_df = pd.DataFrame(search_pairs)
searches_df = pd.DataFrame(searches.values())
searches_df = searches_df.rename(columns={"id": "search_id"})

In [9]:
search_pairs_df = search_pairs_df.merge(searches_df, on="search_id")
search_pairs_df = search_pairs_df.merge(ref_df, on="reference_id")
search_pairs_df

,reference_id,search_id,assessment_id_x,search_type,source,title_x,slug,description,search_string,import_file,...,title_y,authors_short,authors,year,journal,abstract,full_text_url,created_y,last_updated_y,block_id
0,5,5,2,i,1,pmid import,pmid-import,Description,"24004895,21284075,15907334,11778423",,...,Psycho-physiological effects of the terrorist ...,"Kawana N, Ishimatsu S, and Kanda K","Kawana N, Ishimatsu S, Kanda K",2001,Mil Med 2001; 166 (12 Suppl):23-6,The investigation describes the follow-up of t...,,2020-05-08 17:03:52.095604+00:00,2020-05-08 17:04:14.726133+00:00,None
1,6,5,2,i,1,pmid import,pmid-import,Description,"24004895,21284075,15907334,11778423",,...,Effects of bisphenol A-related diphenylalkanes...,Letcher RJ et al.,"Letcher RJ, Sanderson JT, Bokkers A, Giesy JP,...",2005,Toxicol. Appl. Pharmacol. 2005; 209 (2):95-104,The present study investigated the effects of ...,,2020-05-08 17:03:52.203728+00:00,2020-05-08 17:04:17.766639+00:00,None
2,7,5,2,i,1,pmid import,pmid-import,Description,"24004895,21284075,15907334,11778423",,...,An oral developmental neurotoxicity study of d...,Biesemeier JA et al.,"Biesemeier JA, Beck MJ, Silberberg H, Myers NR...",2011,Birth Defects Res. B Dev. Reprod. Toxicol. 201...,"<span class=""abstract_label"">BACKGROUND: </spa...",,2020-05-08 17:03:52.391195+00:00,2020-05-08 17:56:45.324222+00:00,None
3,8,5,2,i,1,pmid import,pmid-import,Description,"24004895,21284075,15907334,11778423",,...,Prenatal folic acid and risk of asthma in chil...,Crider KS et al.,"Crider KS, Cordero AM, Qi YP, Mulinare J, Dowl...",2013,Am. J. Clin. Nutr. 2013; 98 (5):1272-81,"<span class=""abstract_label"">BACKGROUND: </spa...",,2020-05-08 17:03:52.500862+00:00,2020-05-08 17:04:11.036574+00:00,None
4,12,2,2,i,0,Manual import,manual-import,Default search instance used for manually impo...,None. This is used to manually enter literature.,,...,Microbial Ecology of the Human Skin.,Cundell AM,Cundell AM,2018,Microb Ecol 2018; 76 (1):113-120,This review article on the skin microbiota was...,,2022-08-17 15:12:54.897196+00:00,2022-08-17 15:13:13.097661+00:00,None
